In [2]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from lasio import LASFile
import mplcursors
from sklearn.decomposition import PCA
import seaborn as sns
from numpy.linalg import eig

def plot_eigenvectors(eig_vectors, selected_curves):
    fig, ax = plt.subplots()
    for i, eig_vector in enumerate(eig_vectors.T):
        ax.plot(selected_curves, eig_vector, label=f"Autovector {i + 1}")
    ax.set_title("Autovectores de la matriz de covarianza")
    ax.legend()
    plt.show()

def compute_principal_components():
    print("Calculando componentes principales...")
    if las_data is None:
        return

    selected_curves = [var.get() for var in curve_vars if var.get()]
    if not selected_curves or len(selected_curves) < 2:
        return

    try:
        top_depth = float(top_depth_entry.get())
        bottom_depth = float(bottom_depth_entry.get())
    except ValueError:
        print("Por favor, ingrese valores numéricos válidos para el tope y la base.")
        return

    depth_filtered_data = las_data.loc[top_depth:bottom_depth, selected_curves].dropna()
    pca = PCA()
    pca.fit(depth_filtered_data)

    print("\nComponentes principales:")
    print(pca.components_)

    print("\nVarianza explicada:")
    print(pca.explained_variance_ratio_)

    # 1) Matriz de covarianza
    cov_matrix = np.cov(depth_filtered_data.T)
    print("\nMatriz de covarianza:")
    
    # Muestra la matriz de covarianza con etiquetas de curvas
    cov_matrix_df = pd.DataFrame(cov_matrix, index=selected_curves, columns=selected_curves)
    print(cov_matrix_df)
    
    # 1) Matriz de correlación
    corr_matrix = depth_filtered_data.corr()
    print("\nMatriz de correlación:")
    print(corr_matrix)

    # 2) Autovalores y autovectores
    eig_values, eig_vectors = eig(cov_matrix)
    print("\nAutovalores:")
    print(eig_values)
    print("\nAutovectores:")
    print(eig_vectors)
    
   # Añadir esta línea para llamar a la función plot_eigenvectors
    plot_eigenvectors(eig_vectors, selected_curves)  

    
 # Interpretación básica de las curvas seleccionadas en el dominio de PCA
    print("\nInterpretación básica de las curvas seleccionadas en el dominio de PCA:")
    for i, component in enumerate(pca.components_, start=1):
        print(f"Componente principal {i}:")
        max_load_index = np.argmax(np.abs(component))
        max_load_curve = selected_curves[max_load_index]
        print(f"  La curva que aporta la mayor varianza es: {max_load_curve}")
        for curve, load in zip(selected_curves, component):
            print(f"  {curve}: {load:.2f}")
        print()
        
   # Identificar las curvas con mayor correlación entre ellas
    max_corr = 0
    max_corr_pair = None
    for i, curve1 in enumerate(corr_matrix.columns[:-1]):
        for curve2 in corr_matrix.columns[i+1:]:
            corr = corr_matrix[curve1][curve2]
            if abs(corr) > abs(max_corr):
                max_corr = corr
                max_corr_pair = (curve1, curve2)
    print(f"\nLas curvas con mayor correlación entre ellas son {max_corr_pair[0]} y {max_corr_pair[1]}, con un coeficiente de correlación de {max_corr:.2f}\n")        

        
        
    pca_coordinates = pca.transform(depth_filtered_data)  # Añadir esta línea
    # 3) Gráfico de distribución de las coordenadas
    plot_pca_coordinates(pca_coordinates)
    
    # Interpretación básica de las curvas seleccionadas en el dominio de PCA
    print("\nInterpretación básica de las curvas seleccionadas en el dominio de PCA:")
    for i, component in enumerate(pca.components_, start=1):
        print(f"Componente principal {i}:")
        for curve, load in zip(selected_curves, component):
            print(f"  {curve}: {load:.2f}")
        print()


def plot_pca_coordinates(pca_coordinates):
    plt.figure()
    plt.scatter(x=pca_coordinates[:, 0], y=pca_coordinates[:, 1])
    plt.axhline(0, color='red', linestyle='--')
    plt.axvline(0, color='red', linestyle='--')
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.title("Distribución de las coordenadas en el espacio de PC1 y PC2")
    plt.draw()
    plt.show()
    
def open_file():
    file_path = filedialog.askopenfilename(filetypes=[("LAS files", "*.las")])
    if file_path:
        global las_data, las_curves
        las = LASFile(file_path)
        null_value = las.well["NULL"].value
        las_data = las.df().replace(null_value, np.nan)
        las_curves = las.curves
        create_curve_checkbuttons()

def plot_curves():
    if las_data is None:
        return

    selected_curves = [var.get() for var in curve_vars if var.get()]
    if not selected_curves:
        return

    fig, ax = plt.subplots()
    for column in selected_curves:
        ax.plot(las_data[column], las_data.index, label=column)
    ax.invert_yaxis()
    ax.legend()
    #mplcursors.cursor(ax, hover=True)
    plt.show()

def plot_curves_interval():
    if las_data is None:
        return

    selected_curves = [var.get() for var in curve_vars if var.get()]
    if not selected_curves:
        return

    try:
        top_depth = float(top_depth_entry.get())
        bottom_depth = float(bottom_depth_entry.get())
    except ValueError:
        print("Por favor, ingrese valores numéricos válidos para el tope y la base.")
        return

    fig, ax = plt.subplots()
    for column in selected_curves:
        ax.plot(las_data[column], las_data.index, label=column)
    ax.invert_yaxis()
    ax.set_ylim(bottom_depth, top_depth)
    ax.legend()
    #mplcursors.cursor(ax, hover=True)
    plt.show()

def plot_correlation_graph(corr_matrix, title):
    fig, ax = plt.subplots()
    cax = ax.matshow(corr_matrix, cmap='coolwarm')
    fig.colorbar(cax)
    ax.set_xticks(range(len(corr_matrix.columns)))
    ax.set_yticks(range(len(corr_matrix.index)))
    ax.set_xticklabels(corr_matrix.columns)
    ax.set_yticklabels(corr_matrix.index)
    ax.set_title(title)
    plt.show()

def compute_pearson_correlation():
    print("Calculando correlación de Pearson...")
    if las_data is None:
        return

    selected_curves = [var.get() for var in curve_vars if var.get()]
    if not selected_curves or len(selected_curves) < 2:
        return

    try:
        top_depth = float(top_depth_entry.get())
        bottom_depth = float(bottom_depth_entry.get())
    except ValueError:
        print("Por favor, ingrese valores numéricos válidos para el tope y la base.")
        return

    depth_filtered_data = las_data.loc[top_depth:bottom_depth, selected_curves]
    corr_matrix = depth_filtered_data.corr(method="pearson")
    print("Coeficiente de correlación de Pearson entre las curvas seleccionadas en el intervalo de profundidad:")
    print(corr_matrix)

    plot_correlation_graph(corr_matrix, "Coeficiente de correlación de Pearson")

def compute_spearman_correlation():
    print("Calculando correlación de Spearman...")
    if las_data is None:
        return

    selected_curves = [var.get() for var in curve_vars if var.get()]
    if not selected_curves or len(selected_curves) < 2:
        return

    try:
        top_depth = float(top_depth_entry.get())
        bottom_depth = float(bottom_depth_entry.get())
    except ValueError:
        print("Por favor, ingrese valores numéricos válidos para el tope y la base.")
        return

    depth_filtered_data = las_data.loc[top_depth:bottom_depth, selected_curves]
    corr_matrix = depth_filtered_data.corr(method="spearman")
    print("Coeficiente de correlación de Spearman entre las curvas seleccionadas en el intervalo de profundidad:")
    print(corr_matrix)

    plot_correlation_graph(corr_matrix, "Coeficiente de correlación de Spearman")



def create_curve_checkbuttons():
    for child in curve_frame.winfo_children():
        child.destroy()

    global curve_vars
    curve_vars = []
    for curve in las_curves:
        if curve.mnemonic == "DEPT":
            continue
        var = tk.StringVar()
        curve_vars.append(var)
        checkbutton = tk.Checkbutton(curve_frame, text=curve.mnemonic, variable=var, onvalue=curve.mnemonic, offvalue="")
        checkbutton.pack()

las_data = None
las_curves = None
curve_vars = []

root = tk.Tk()
root.title("Coeficientes de Correlación")

open_button = tk.Button(root, text="Abrir archivo LAS", command=open_file)
open_button.pack()

plot_button = tk.Button(root, text="Mostrar curvas", command=plot_curves)
plot_button.pack()

curve_frame = tk.Frame(root)
curve_frame.pack()

top_depth_label = tk.Label(root, text="Profundidad del tope:")
top_depth_label.pack()
top_depth_entry = tk.Entry(root)
top_depth_entry.pack()

bottom_depth_label = tk.Label(root, text="Profundidad de la base:")
bottom_depth_label.pack()
bottom_depth_entry = tk.Entry(root)
bottom_depth_entry.pack()

plot_interval_button = tk.Button(root, text="Mostrar curvas en el intervalo", command=plot_curves_interval)
plot_interval_button.pack()

compute_pearson_button = tk.Button(root, text="Calcular coeficiente de correlación de Pearson", command=compute_pearson_correlation, bg="darkblue", fg="white")
compute_pearson_button.pack()

compute_spearman_button = tk.Button(root, text="Calcular coeficiente de correlación de Spearman", command=compute_spearman_correlation, bg="darkblue", fg="white")
compute_spearman_button.pack()

compute_pca_button = tk.Button(root, text="Calcular componentes principales", command=compute_principal_components, bg="darkblue", fg="white")
compute_pca_button.pack()

root.mainloop()

Calculando correlación de Pearson...
Coeficiente de correlación de Pearson entre las curvas seleccionadas en el intervalo de profundidad:
          NPHI      RHOB
NPHI  1.000000 -0.718083
RHOB -0.718083  1.000000
Calculando correlación de Spearman...
Coeficiente de correlación de Spearman entre las curvas seleccionadas en el intervalo de profundidad:
          NPHI      RHOB
NPHI  1.000000 -0.727506
RHOB -0.727506  1.000000
Calculando componentes principales...

Componentes principales:
[[ 0.79569017 -0.60570385]
 [-0.60570385 -0.79569017]]

Varianza explicada:
[0.86536847 0.13463153]

Matriz de covarianza:
          NPHI      RHOB
NPHI  0.013387 -0.007894
RHOB -0.007894  0.009027

Matriz de correlación:
          NPHI      RHOB
NPHI  1.000000 -0.718083
RHOB -0.718083  1.000000

Autovalores:
[0.01939642 0.00301764]

Autovectores:
[[ 0.79569017  0.60570385]
 [-0.60570385  0.79569017]]

Interpretación básica de las curvas seleccionadas en el dominio de PCA:
Componente principal 1:
  La c